# Liste des fonctions:

In [27]:
'''
# permet d'écouter un fichier wav
Ecoute_Audio(audio_path):
# duration signal:
duration(audio_path)

# Permet de retourner un spectre en utilisant librosa depuis un path
stft_path(audio_path,part=10,cross=0.1)
# Permet de tracer le plot d'un spectre en utilisant librosa et seaborn
plot_spectrogram(audio, fe, dt=0.025)
# Permet de tracer le plot d'un spectre en utilisant librosa depuis un path
plot_spectrogram_path(audio_path, dt=0.025)

logMelSpectrogram(audio, fe, dt)
# Permet de retourner un spectre Log Mel en utilisant librosa depuis un Path
logMelSpectrogram_path(audio_path, dt=0.025)
# Permet de tracer le plot d'un spectre en utilisant librosa depuis un Path
plot_logMelSpectrogram_path(audio_path, dt=0.025)

# Permet de tracer un spectre en utilisant librosa depuis un array data/fe
spectrogram(audio,fe,dt=0.025)
# Permet de retourner un spectre Log Mel en utilisant librosa depuis un array data/fe
logMelSpectrogram(audio, fe, dt=0.025)
# Permet de tracer le plot d'un spectre en utilisant librosa depuis un array data/fe
plot_logMelSpectrogram(audio, fe, dt=0.025)

# permet de lire et de récupérer les données d'un fichier wav
wave_path(machine,folder,type_audio,k)
# permet de faire la transformée de Fourier et de tracer le spectre d'un signal
tracerSpectre_op(data,rate,debut,duree)
#permet de faire la transformée de fourrier sur le signal
Fourier(data,rate,debut,duree)
#permet de faire la transformée de fourrier sur le signal
Frequences(data,rate)

#Algorythme de parcours de fichiers
Dossier_data(DOSSIER)
#Extraction de tous les spectres
Extraction_spectre(dataset,col_URL,col_data,on_data)
Extraction_spectreV2(dataset,col_URL,col_data,on_data)

# ajout des données dans un Dataframe
Add_spectre_df(dataset,Machine,rate,spectre)

#Générateur de spectre
class spectrum_generator()

#création du masque
class downsizing()

'''

"\n# permet d'écouter un fichier wav\nEcoute_Audio(audio_path):\n# duration signal:\nduration(audio_path)\n\n# Permet de retourner un spectre en utilisant librosa depuis un path\nstft_path(audio_path,part=10,cross=0.1)\n# Permet de tracer le plot d'un spectre en utilisant librosa et seaborn\nplot_spectrogram(audio, fe, dt=0.025)\n# Permet de tracer le plot d'un spectre en utilisant librosa depuis un path\nplot_spectrogram_path(audio_path, dt=0.025)\n\nlogMelSpectrogram(audio, fe, dt)\n# Permet de retourner un spectre Log Mel en utilisant librosa depuis un Path\nlogMelSpectrogram_path(audio_path, dt=0.025)\n# Permet de tracer le plot d'un spectre en utilisant librosa depuis un Path\nplot_logMelSpectrogram_path(audio_path, dt=0.025)\n\n# Permet de tracer un spectre en utilisant librosa depuis un array data/fe\nspectrogram(audio,fe,dt=0.025)\n# Permet de retourner un spectre Log Mel en utilisant librosa depuis un array data/fe\nlogMelSpectrogram(audio, fe, dt=0.025)\n# Permet de tracer le

# Fonctions

## Génération Spectre

In [28]:
# Permet de tracer un spectre en utilisant librosa depuis un array data/fe
def spectrogram(audio,fe,dt=0.025):
    return np.abs(librosa.stft(y=audio,n_fft=int(dt*fe),hop_length=int(dt*fe)))


In [29]:
#Générateur de spectre
class spectrum_generator():
    def __init__(self):
        self.level=0
        self.cut=None
        self.nb_freq=0
        self.nb_dt=0
        self.nb_samples_tot=0
        self.nb_samples_target=0
        
    def fit(self,X,y,level=0,dt_cut=None):
        #récupération des paramètres
        self.level=level
        self.nb_samples_tot,self.nb_dt,self.nb_freq=X.shape
        
        if dt_cut=='first':
            self.cut=0
        elif dt_cut=='last':
            self.cut=-1
        elif dt_cut=='both':  
            self.cut=[0,-1]
        else :
            self.cut=None
        
    def flow_from_numpy(self,X,y,batchsize=32,pourcent_modif=0.1):
        #initialisation de la selection aléatoire des samples
        selector=np.random.random(batchsize)*self.nb_samples_tot
        selector=selector.astype(int)
        transform_sel=np.random.random(int(pourcent_modif*batchsize))*selector.shape
        transform_sel=transform_sel.astype(int)
        #sélection des samples
        X_selected=np.array(X)[selector]
        y_selected=np.array(y)[selector]
        #transformation aléatoire des samples via une des fonctions
        for modif in transform_sel:
            choix_modif=int(np.random.random(1)*3)
            if choix_modif<=1:
                X_selected[modif,:,:]=X_selected[modif,:,:]*(1+self.level)
           
            elif (choix_modif>1) and (choix_modif<=2):
                X_selected[modif,:,:]=X_selected[modif,:,:]*(1-self.level)  
            
            elif (choix_modif>2) and (choix_modif<=3):
                for dt in self.cut:
                    X_selected[modif,dt,:]=X_selected[modif,dt,:]*0
            else:
                X_selected[modif,:,:]=X_selected[modif,:,:]
        
        return X_selected,y_selected
        
    def add_samples(self,X,y,pourcent_add=0.1, labels='all'):
        self.nb_samples_tot=y.shape[0]
        if labels=='all':
            self.nb_samples_target=self.nb_samples_tot
        else:
            self.nb_samples_target=y[y==labels].shape[0]
        
        if self.nb_samples_target ==self.nb_samples_tot : 
            #initialisation de la selection aléatoire des samples
            selector=np.random.random(int(pourcent_add*self.nb_samples_tot))*self.nb_samples_tot
            selector=selector.astype(int)
            #sélection des samples
            X_selected=np.array(X)[selector]
            y_selected=np.array(y)[selector]
        
        else :
            selector=np.random.random(int(pourcent_add*self.nb_samples_target))*self.nb_samples_target
            selector=selector.astype(int)
            #sélection des samples
            X_selected=np.array(X)[y==labels][selector]
            y_selected=np.array(y)[y==labels][selector]
        
        #transformation aléatoire des samples via une des fonctions
        for modif in range(X_selected.shape[0]):
            choix_modif=int(np.random.random(1)*3)
            if choix_modif<=1:
                X_selected[modif,:,:]=X_selected[modif,:,:]*(1+self.level)
           
            elif (choix_modif>1) and (choix_modif<=2):
                X_selected[modif,:,:]=X_selected[modif,:,:]*(1-self.level)  
            
            elif (choix_modif>2) and (choix_modif<=3):
                for dt in self.cut:
                    X_selected[modif,dt,:]=X_selected[modif,dt,:]*0
            else:
                X_selected[modif,:,:]=X_selected[modif,:,:]
        
        return np.concatenate((X,X_selected),axis=0),np.concatenate((y,y_selected),axis=0)   
        

In [30]:
# Permet de retourner un spectre en utilisant librosa depuis un path
def stft_path(audio_path,part=10,cross=0.1):
    """
    part: int défini le nombre de partitions à créer dans le fichier 
    cross: % défini le chevauchement entre partitions
    """
    audio,fe=librosa.load(audio_path)
    rate=fe
    n_fft=int(len(audio)/part*(1+cross))
    hop_length=int(len(audio)/part*(1-cross))
    spectre=np.abs(librosa.stft(y=audio,n_fft=n_fft,hop_length=hop_length))
    return rate,spectre

In [31]:
#chargement des données en entrée
def load_data(audio_path, dt=0.5, T_max=10, normalize=False):
    """
    Permet de charger un spectre logmel depuis un path. 
    
    params: 
    
    dt: float /int. si [0:1[ alors dt est considéré comme un pas de découpage en seconde 
        si >=1 alors doit être un int réprésentant le nombre de découpages souhaité
    
    """
    X_audio=[]
    nb_boucles=len(audio_path)
    pourcent=nb_boucles//100
    tps1 = time.time()
    print("démarrage") 
    
    if dt<1: # on veut selectionner le pas en seconde

        for i, a_path in enumerate(audio_path) :

            # Load the audio file
            data, fe = librosa.load(a_path, sr=None)

            # For the audio > T_max : Use just the fist T_max seconde, to have the right shape. 
            if len(data)>= T_max*fe:
                data = data[:int(T_max*fe)]

            # For the audio < T_max : Add in the signal a zeros vector, to have the right shape.
            else :
                data = np.concatenate([data, np.zeros(int(T_max*fe - len(data)))])

            # Apply the logMelSpectrogram function.    
            spectre_audio = logMelSpectrogram(data, fe, dt)
            X_audio.append(spectre_audio)
            if i%pourcent==0 or i==0:
                clear_output()
                print('dimension des données de sortie',np.array(X_audio).shape)
                tps2 = time.time()
                print('boucle: ',i+1 ,' / ', nb_boucles+1)
                print('temps écoulé:{:.2f} sec'.format(tps2 - tps1))
                print('temps restant:{:.0f} min'.format((tps2 - tps1)*(nb_boucles-i)/((i+1)*60)))

        tps3 = time.time() 
        print('temps total:',tps3 - tps1)
        
        
    else: # on veut selectionner un nombre de dt 
       
        for i, a_path in enumerate(audio_path) :

            # Load the audio file
            data, fe = librosa.load(a_path, sr=None)

            # For the audio > T_max : Use just the fist T_max seconde, to have the right shape. 
            if len(data)>= T_max*fe:
                data = data[:int(T_max*fe)]

            # For the audio < T_max : Add in the signal a zeros vector, to have the right shape.
            else :
                data = np.concatenate([data, np.zeros(int(T_max*fe - len(data)))])

            # Apply the logMelSpectrogram function. 
            nb_dot= len(data)//dt
            spectre_audio = logMelSpectrogram(data[:nb_dot*dt], fe, nb_dot)
            X_audio.append(spectre_audio)
            if i%pourcent==0 or i==0:
                clear_output()
                print('dimension des données de sortie',np.array(X_audio).shape)
                tps2 = time.time()
                print('boucle: ',i+1 ,' / ', nb_boucles+1)
                print('temps écoulé:{:.2f} sec'.format(tps2 - tps1))
                print('temps restant:{:.0f} min'.format((tps2 - tps1)*(nb_boucles-i)/((i+1)*60)))

        tps3 = time.time() 
        print('temps total:',tps3 - tps1)
    
    if normalize : 
        return (np.array(X_audio)-np.array(X_audio).min())/(np.array(X_audio).max()-np.array(X_audio).min())
    else :
        return np.array(X_audio)

In [32]:
# Permet de tracer un spectre en utilisant librosa
def spectrogram(audio,fe,dt):
    return np.abs(librosa.stft(y=audio,n_fft=int(dt*fe),hop_length=int(dt*fe)))

In [33]:
# Permet de tracer un spectre en utilisant librosa
def spectrogram_path(audio_path,dt):
    audio,fe=librosa.load(audio_path)
    return np.abs(librosa.stft(y=audio,n_fft=int(dt*fe),hop_length=int(dt*fe)))


In [34]:
# Permet de retourner un spectre Log Mel en utilisant librosa depuis un Path
def logMelSpectrogram_path(audio_path, dt=0.025):
    audio,fe=librosa.load(audio_path)
    stfts = np.abs(librosa.stft(audio,n_fft = int(dt*fe),hop_length = int(dt*fe),center = True)).T
    num_spectrogram_bins = stfts.shape[-1]
    # MEL filter
    linear_to_mel_weight_matrix = librosa.filters.mel(sr=fe,n_fft=int(dt*fe) + 1,n_mels=num_spectrogram_bins,).T
    # Apply the filter to the spectrogram
    mel_spectrograms = np.tensordot(stfts,linear_to_mel_weight_matrix,1)
    return np.log(mel_spectrograms + 1e-6)

In [35]:
# Permet de retourner un spectre Log Mel en utilisant librosa depuis un array data/fe
def logMelSpectrogram(audio, fe, dt=0.025):
    # Spectrogram
    stfts = np.abs(librosa.stft(audio,
                        n_fft = int(dt*fe),
                        hop_length = int(dt*fe),
                        center = True
                        )).T
    num_spectrogram_bins = stfts.shape[-1]
    
    # MEL filter
    linear_to_mel_weight_matrix = librosa.filters.mel(
                                sr=fe,
                                n_fft=int(dt*fe) + 1,
                                n_mels=num_spectrogram_bins,
                    ).T

    # Apply the filter to the spectrogram
    mel_spectrograms = np.tensordot(
                stfts,
                linear_to_mel_weight_matrix,
                1
            )
    return np.log(mel_spectrograms + 1e-6)

In [36]:
#permet de faire la transformée de fourrier sur le signal
def Fourier(data,rate,debut,duree):
    start = int(debut*rate)
    stop = int((debut+duree)*rate)
    # Récupération de la valeur absolue de la transformée de fourier sur l'échantillon
    spectre = np.absolute(fft(data[start:stop]))
    # normalisation des valeurs 0/1
    spectre = spectre/spectre.max()
    # On crée les labels des abscices 
    n = spectre.size
    freq=np.linspace(1/n,1,n)*rate
    return freq,spectre


In [37]:
#permet de faire la transformée de fourrier sur le signal
def Frequences(data,rate):
    n = data.size
    duree = 1.0*n/rate
    start = int(0)
    stop = int(duree*rate)
    # Récupération de la valeur absolue de la transformée de fourier sur l'échantillon
    spectre = np.absolute(fft(data[start:stop]))
    # normalisation des valeurs 0/1
    spectre = spectre/spectre.max()
    return spectre


In [38]:
def logMelSpectrogram(audio, fe, dt):
    stfts = np.abs(librosa.stft(audio,n_fft = int(dt*fe),hop_length = int(dt*fe),center = True)).T
    num_spectrogram_bins = stfts.shape[-1]
    # MEL filter
    linear_to_mel_weight_matrix = librosa.filters.mel(sr=fe,n_fft=int(dt*fe) + 1,n_mels=num_spectrogram_bins,).T
    # Apply the filter to the spectrogram
    mel_spectrograms = np.tensordot(stfts,linear_to_mel_weight_matrix,1)
    return np.log(mel_spectrograms + 1e-6)

## Reduction de données

In [39]:
#création du masque
class downsizing():
    def __init__(self):
        self.selector=0
        self.datamax=0
    
    def fit(self,X):
        self.datamax=X.max(axis=1).max(axis=0)
    
    def transform(self,X,seuil_db):
        if seuil_db=='brute':
            #Méthode brute
            data_s=X.sum(axis=1).sum(axis=0)
            data_s.shape
            self.selector=np.where(data_s>0,True,False)  
        else:  
            self.selector=np.where(self.datamax>seuil_db,True,False)

        return X[:,:,self.selector]
    
    def get_selector(self,seuil_db):
        self.selector=np.where(self.datamax>seuil_db,True,False)
        return self.selector

## Plotting

In [40]:
# Permet de tracer le plot d'un spectre en utilisant librosa et seaborn
def plot_spectrogram(audio, fe, dt=0.025):
    im = spectrogram(audio, fe, dt)
    sns.heatmap(np.rot90(im.T), cmap='inferno', vmin=0, vmax=np.max(im)/3)
    loc, labels = plt.xticks()
    l = np.round((loc-loc.min())*len(audio)/fe/loc.max(), 2)
    plt.xticks(loc, l)
    loc, labels = plt.yticks()
    l = np.array(loc[::-1]*fe/2/loc.max(), dtype=int)
    plt.yticks(loc, l)
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Hz)")

In [41]:
# Permet de tracer le plot d'un spectre en utilisant librosa depuis un path
def plot_spectrogram_path(audio_path, dt=0.025):
    audio,fe=librosa.load(audio_path)
    im = spectrogram_path(audio_path,dt)
    sns.heatmap(np.rot90(im.T), cmap='inferno', vmin=0, vmax=np.max(im)/3)
    loc, labels = plt.xticks()
    l = np.round((loc-loc.min())*len(audio)/fe/loc.max(), 2)
    plt.xticks(loc, l)
    loc, labels = plt.yticks()
    l = np.array(loc[::-1]*fe/2/loc.max(), dtype=int)
    plt.yticks(loc, l)
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Hz)")

In [42]:
# Permet de tracer le plot d'un spectre en utilisant librosa depuis un Path
def plot_logMelSpectrogram_path(audio_path, dt=0.025):
    audio,fe=librosa.load(audio_path)
    sns.heatmap(np.rot90(logMelSpectrogram(audio, fe, dt)), cmap='inferno', vmin = -6)
    loc, labels = plt.xticks()
    l = np.round((loc-loc.min())*len(audio)/fe/loc.max(), 2)
    plt.xticks(loc, l)
    plt.yticks([])
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Mel)")

In [43]:
# Permet de tracer le plot d'un spectre en utilisant librosa depuis un array data/fe
def plot_logMelSpectrogram(audio, fe, dt=0.025):
    sns.heatmap(np.rot90(logMelSpectrogram(audio, fe, dt)), cmap='inferno', vmin = -6)
    loc, labels = plt.xticks()
    l = np.round((loc-loc.min())*len(audio)/fe/loc.max(), 2)
    plt.xticks(loc, l)
    plt.yticks([])
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Mel)")

In [44]:
# permet de faire la transformée de Fourier et de tracer le spectre d'un signal
def tracerSpectre_op(data,rate,debut,duree):
    start = int(debut*rate)
    stop = int((debut+duree)*rate)
    # Récupération de la valeur absolue de la transformée de fourier sur l'échantillon
    spectre = np.absolute(fft(data[start:stop]))
    # normalisation des valeurs 0/1
    spectre = spectre/spectre.max()
    # On crée les labels des abscices 
    n = spectre.size
    freq=np.linspace(1/n,1,n)*rate
    # On trace un barplot avec x=freq,y=spectre
    plt.vlines(freq,[0],spectre,'r')
    plt.xlabel('f (Hz)')
    plt.ylabel('A')
    plt.axis([0,0.5*rate,0,1])
    plt.grid()

## Autres

In [45]:
# permet de lire et de récupérer les données d'un fichier wav
def wave_path(machine,folder,type_audio,k):
    machine_path='./'+machine+'/'
    type_path=folder+'/'
    zeros='00000000'+str(k)
    file_name=type_audio+'_id_00_'+zeros[-8:]+'.wav'
    file_path=machine_path+type_path+file_name
    rate,data = wave.read(file_path)
    graph_name=file_name.split('.')[0]+'_spectre.jpg'
    save_path=machine_path+'Spectres/'+graph_name
    n = data.size
    duree = 1.0*n/rate
    return file_path,save_path,n,duree,rate,data

In [46]:
#Algorythme de parcours de fichiers

def Dossier_data(DOSSIER):
    """
    Cette fonction permet de parcourir les fichiers contenu 
    dans le DOSSIER 'DOSSIER' QUE VOUS SOUHAITEZ du notebook d'en extraire
    les informations et de générer un PandaDataFrame
    """
    chemin=glob.glob(''+DOSSIER+'/**',recursive=True)
    df=pd.DataFrame(chemin, columns=['URL'])
    df['URL2']=df['URL'].apply(lambda contenu: os.path.split(contenu)[0])
    df['fichier']=df['URL'].apply(lambda contenu: os.path.split(contenu)[1])
    df['train_test']=df['URL2'].apply(lambda contenu: os.path.split(contenu)[1])
    df['URL2']=df['URL2'].apply(lambda contenu: os.path.split(contenu)[0])
    df['Machine']=df['URL2'].apply(lambda contenu: os.path.split(contenu)[1])
    df=df[(df['Machine']!='')&(df['Machine']!='.')&(df['Machine']!='Datascience_projet_son')&(df['Machine']!='./archive')&(df['URL2']!='./archive')&(df['URL2']!='archive')]
    df['Machine_ID']=df['fichier'].apply(lambda contenu: contenu.split('_')[2])
    df['son_type']=df['fichier'].apply(lambda contenu: contenu.split('_')[0])
    df=df.drop('URL2',axis=1)
    duree=[]
    nb_boucles=df.shape[0]
    tps1 = time.time()
    for i,URL in enumerate(df['URL']):
      tps2 = time.time()
      print('boucle: ',i+1 ,' / ', nb_boucles+1)
      print('temps écoulé:{:.2f} sec'.format(tps2 - tps1))
      print('temps restant:{:.0f} min'.format((tps2 - tps1)*(nb_boucles-i)/((i+1)*60)))
      duree.append(duration(URL))
      clear_output()
      if i%1000==0 : # on supprime le cache de la RAM toutes les 1000 itérations
        librosa.cache.clear()

    df['duree_son']=duree
    tps3 = time.time()
    print('temps total:',tps3 - tps1)
    return df

In [47]:
###Extraction de tous les spectres

def Extraction_spectre(dataset,col_URL,col_data,on_data):
    """
    Cette fonction permet d'extraire les spectre d'une liste de fichier contenu dans un DataFrame
    
    Inputs : 
    dataset : DataFrame, fichier à regarder
    col_URL : str, nom de la colone contenant l'URL du fichier
    col_data: str, nom de la colone contenant le filtre à utiliser 
    on_data: str/int,float  contenu du filtre à utiliser (col_data= colonne 1, on_data=Fan ne gardera que les observations Fan de la colonne 1)
    
    
    Outputs : rate_list, spectre_list
    rate_list : Liste contenant la fréquence d'échantillonage du son (1 fréquence par fichier)
    spectre_list : Liste contenant la liste des fréquences normalisées du fichier (rate_list fréquences par fichier)
    
    """
    rate_list=[]
    spectre_list=[]
    tps1 = time.time()
    data=dataset[(dataset[col_data]==on_data)&(dataset['fichier_type']=='wav')][col_URL]
    nb_boucles=data.shape[0]
    
    for i, elements in enumerate(data):
        tps2 = time.time()
        print('boucle: ',i+1 ,' / ', nb_boucles+1)
        print('temps écoulé:{:.2f} sec'.format(tps2 - tps1))
        print('temps restant:{:.0f} min'.format((tps2 - tps1)*(nb_boucles-i)/((i+1)*60)))
        rate,data = wave.read(elements)
        spectre=Frequences(data,rate)
        rate_list.append(rate)
        spectre_list.append(spectre)
        clear_output()
    
    tps3 = time.time()
    print('temps total:',tps3 - tps1)
    return rate_list,spectre_list

In [48]:
def Extraction_spectreV2(dataset,col_URL,col_data,on_data):
    """
    Cette fonction permet d'extraire les spectre d'une liste de fichier contenu dans un DataFrame et enregistrer directement dans le dataframe
    
    Inputs : 
    dataset : DataFrame, fichier à regarder
    col_URL : str, nom de la colone contenant l'URL du fichier
    col_data: str, nom de la colone contenant le filtre à utiliser 
    on_data: str/int,float  contenu du filtre à utiliser (col_data= colonne 1, on_data=Fan ne gardera que les observations Fan de la colonne 1)
    
    
    Outputs : rate_list, spectre_list
    rate_list : Liste contenant la fréquence d'échantillonage du son (1 fréquence par fichier)
    spectre_list : Liste contenant la liste des fréquences normalisées du fichier (rate_list fréquences par fichier)
    
    """

    tps1 = time.time()
    df_new=dataset
    data=dataset[(dataset[col_data]==on_data)&(dataset['fichier_type']=='wav')][col_URL]
    lignes_select=(dataset[col_data]==on_data)&(dataset['fichier_type']=='wav')
    nb_boucles=data.shape[0]
    rate=[]
    spectre=[]    
    for i, elements in enumerate(data):
        tps2 = time.time()
        print('boucle: ',i+1 ,' / ', nb_boucles+1)
        print('temps écoulé:{:.2f} sec'.format(tps2 - tps1))
        print('temps restant:{:.0f} min'.format((tps2 - tps1)*(nb_boucles-i)/((i+1)*60)))
        rate,data = wave.read(elements)
        spectre=Frequences(data,rate)
        clear_output()
    
    df_new.loc[lignes_select,['rate']]=rate
    df_new.loc[lignes_select,['spectre']]=spectre
    
    tps3 = time.time()
    print('temps total:',tps3 - tps1)
    return df_new

In [49]:
# ajout des données dans un Dataframe
def Add_spectre_df(dataset,Machine,rate,spectre):
    """
    Permet d'ajouter une liste de spectre et de rate à un dataset existant
    """
    df_new=dataset # récupération des données existantes
    df_index=pd.DataFrame()
    df_index['num_ligne']=df_new[(df_new['Machine']==Machine)&(df_new['fichier_type']=='wav')].index # récupération des index des données
    df_index['rate']=rate # ajout des données
    df_index['spectre']=spectre
    df_index=df_index.set_index('num_ligne')
    df_new=df_new.join(df_index,how='left') # fusion des nouvelles données dans le dataset sauvegardé
    return df_new


In [50]:
# permet d'écouter un fichier wav
def Ecoute_Audio(audio_path):
    audio,fe=librosa.load(audio_path)
    Audio(audio, rate=fe)

In [51]:
# duration signal:
def duration(audio_path):
    data ,sample_rate =  librosa.load(audio_path, sr=None)
    # librosa.cache.clear()
    len_data = len(data)
    t = len_data / sample_rate
    return t

# Convert To py

In [52]:
# !jupyter nbconvert --to python functions_utils.ipynb
